In [1]:
# experiment testing alignment accuracuy with and without annotation

# take random HIV genomes
# simulate RNA-seq reads using the corresponding annotation
# align to the HXB2 with and without the reference annotation

In [2]:
import os
import re
import sys
import csv
import time
import random
import requests
import subprocess
from pathlib import Path

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from Bio import SeqIO

In [ ]:
data_dir = Path.cwd().parent.parent / 'HIV_Atlas_Creation' / 'data'

sequence_dir = data_dir / 'sequences'
assert sequence_dir.exists(), f"sequence_dir does not exist: {sequence_dir}"

annotation_dir = data_dir / 'annotation'
assert annotation_dir.exists(), f"annotation_dir does not exist: {annotation_dir}"

reference_fasta_fname = data_dir / 'reference' / 'K03455.1.fasta'
reference_gtf_fname = data_dir / 'reference' / 'K03455.1.gtf'
# reference_fasta_fname = sequence_dir / 'K03454.fasta'
# reference_gtf_fname = annotation_dir / 'K03454/K03454.vira.gtf'

base_dir = Path.cwd().parent

outdir = base_dir / 'results' / 'alignment'
outdir.mkdir(parents=True, exist_ok=True)

In [4]:
# software links
soft_dir = base_dir / 'soft'

In [5]:
%load_ext autoreload
%autoreload 1

sys.path.insert(0, str(soft_dir / "genomic_scripts"))
%aimport definitions

In [6]:
random.seed(42)
np.random.seed(42)

In [7]:
# When simmulating reads for alignment experiment - we do not need to do much
# 1. build HXB2 HISAT and STAR indices with and without annotation
# 2. extract transcript fasta from the HXB2 annotation
# 3. Pick a set of random genomes
# 4. for every genome
# 6. extract 100bp reads from each transcript. Write transcriptomic start position of each read into the read name
# 7. align to HXB2 with and without annotation using both HISAT2 and STAR
# 8. for each read, check if the alignment is correct. Comp;ute the number of positions that are in the correct position vs incorrect
# 9. plot the results

In [8]:
# build indices
idx_dir = outdir / 'indices'
idx_dir.mkdir(parents=True, exist_ok=True)

hisat_idx_dir = idx_dir / 'hisat2'
hisat_idx_dir.mkdir(parents=True, exist_ok=True)

# build HISAT2 index
hisat_idx = hisat_idx_dir / 'HXB2'
hisat_cmd = f"hisat2-build {reference_fasta_fname} {hisat_idx}"
print(hisat_cmd)
subprocess.run(hisat_cmd, shell=True, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

hisat_idx_annot = hisat_idx_dir / 'HXB2_with_annotation'
hisat_cmd_annot = f"hisat2_extract_exons.py {reference_gtf_fname} > {hisat_idx_annot}.exons"
print(hisat_cmd_annot)
subprocess.run(hisat_cmd_annot, shell=True, check=True)

hisat_cmd_annot = f"hisat2_extract_splice_sites.py {reference_gtf_fname} > {hisat_idx_annot}.ss"
print(hisat_cmd_annot)
subprocess.run(hisat_cmd_annot, shell=True, check=True)

hisat_cmd_annot = f"hisat2-build --ss {hisat_idx_annot}.ss --exon {hisat_idx_annot}.exons {reference_fasta_fname} {hisat_idx_annot}"
print(hisat_cmd_annot)
subprocess.run(hisat_cmd_annot, shell=True, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# minimap2 - extract bed12 from gtf for the junc-bed
minimap2_idx_dir = idx_dir / 'minimap2'
minimap2_idx_dir.mkdir(parents=True, exist_ok=True)

minimap2_idx = minimap2_idx_dir / 'HXB2'
reference_junc_bed_fname = minimap2_idx_dir / 'HXB2.junc.bed'
cmd = ["paftools.js","gff2bed","-j",reference_gtf_fname]
with open(reference_junc_bed_fname,"w+") as outFP:
    subprocess.call(cmd,stdout=outFP)

hisat2-build /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Creation/data/reference/K03455.1.fasta /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Experiments/results/alignment/indices/hisat2/HXB2
hisat2_extract_exons.py /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Creation/data/reference/K03455.1.gtf > /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Experiments/results/alignment/indices/hisat2/HXB2_with_annotation.exons
hisat2_extract_splice_sites.py /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Creation/data/reference/K03455.1.gtf > /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Experiments/results/alignment/indices/hisat2/HXB2_with_annotation.ss
hisat2-build --ss /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Experiments/results/alignment/indices/hisat2/HXB2_with_annotation.ss --exon /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Experiments/results/alignment/indices/hisat2/HXB2_with_annotation.exons /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Creation/data/reference/K03455.1.fasta /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas

In [9]:
# simulate reads into memory as dict of read sequence to transcript start position and transcript id
def get_read_chain(tx_chain, tx_start_pos, tx_end_pos, strand):
    genome_start_pos = definitions.trans2genome(tx_chain, strand, tx_start_pos)
    genome_end_pos = definitions.trans2genome(tx_chain, strand, tx_end_pos)

    # cut chain to the start/end positions
    read_chain = definitions.cut(tx_chain, genome_start_pos, genome_end_pos)
    return read_chain


def simulate_reads(tx_fasta_fname, tx_chains, read_length=100):
    reads = {}
    for record in SeqIO.parse(tx_fasta_fname, 'fasta'):
        tx_id = record.id
        tx_seq = str(record.seq)
        for i in range(len(tx_seq) - read_length):
            # retrieve genome positions for the read start to end including any introns
            read_genome_chain = get_read_chain(tx_chains[tx_id][1],i,i+read_length,"+")
            # convert to string
            read_genome_chain = ",".join(["-".join([str(y) for y in x]) for x in read_genome_chain])
            read = tx_seq[i:i+read_length]
            read_name = f"{tx_id}/{i}-{i+read_length}"+"/"+read_genome_chain
            reads[read_name] = (read, i, tx_id)
    return reads

In [10]:
# get a subset of genomes to run simulations with
genomes = list(sequence_dir.glob("*.fasta"))
genomes = random.sample(genomes, 10)
# extract genome IDs
genomes = [x.stem for x in genomes]
genomes = ["K03454"]

for genome in genomes:
    genome_fasta_fname = sequence_dir / f"{genome}.fasta"
    genome_gtf_fname = annotation_dir/ genome / f"{genome}.vira.gtf"

    # extract transcripts with gffread
    files_dir = outdir / 'files'
    files_dir.mkdir(parents=True, exist_ok=True)
    genome_dir = files_dir / genome
    genome_dir.mkdir(parents=True, exist_ok=True)

    tx_fasta_fname = genome_dir / f"{genome}.transcripts.fasta"
    gffread_cmd = f"gffread {genome_gtf_fname} -g {genome_fasta_fname} -w {tx_fasta_fname}"
    subprocess.run(gffread_cmd, shell=True, check=True)

    # load transcript chains to later use as translation to genomic coordinates
    tx_chains = definitions.get_chains(genome_gtf_fname,"exon",True)
    # extract chains into a dictionary where key is tid and value is seqid and chain. There is just one chain per tid
    tx_chains = tx_chains.groupby("tid").apply(lambda x: x[["seqid","chain"]].values[0]).to_dict()

    reads = simulate_reads(tx_fasta_fname, tx_chains)

    reads_fname = genome_dir / f"{genome}.reads.fasta"
    with open(reads_fname, 'w') as outFP:
        for read_name, read in reads.items():
            outFP.write(f">{read_name}\n{read[0]}\n")

/tmp/ipykernel_1565720/4175875301.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tx_chains = tx_chains.groupby("tid").apply(lambda x: x[["seqid","chain"]].values[0]).to_dict()


In [13]:
# now we need to run alignment for each genome with and without annotation
for genome in genomes:
    genome_fasta_fname = sequence_dir / f"{genome}.fasta"
    genome_gtf_fname = annotation_dir/ genome / f"{genome}.vira.gtf"
    genome_dir = files_dir / genome
    reads_fname = genome_dir / f"{genome}.reads.fasta"

    hisat_sam_fname = genome_dir / f"{genome}.hisat.woa.sam"
    hisat_cmd = f"hisat2 --score-min L,0,-2 --mp 2,2 -x {hisat_idx} -f -U {reads_fname} -S {hisat_sam_fname}"
    print(hisat_cmd)
    subprocess.run(hisat_cmd, shell=True, check=True)

    hisat_sam_annot_fname = genome_dir / f"{genome}.hisat.wa.sam"
    hisat_cmd_annot = f"hisat2 --score-min L,0,-2 --mp 2,2 -x {hisat_idx_annot} -f -U {reads_fname} -S {hisat_sam_annot_fname}"
    print(hisat_cmd_annot)
    subprocess.run(hisat_cmd_annot, shell=True, check=True)

    # map with minimap2
    minimap2_sam_fname = genome_dir / f"{genome}.minimap2.woa.sam"
    minimap2_cmd = f"minimap2 -a -x splice {reference_fasta_fname} {reads_fname} > {minimap2_sam_fname}"
    print(minimap2_cmd)
    subprocess.run(minimap2_cmd, shell=True, check=True)

    minimap2_sam_annot_fname = genome_dir / f"{genome}.minimap2.wa.sam"
    minimap2_cmd_annot = f"minimap2 --junc-bed {reference_junc_bed_fname} -a -x splice {reference_fasta_fname} {reads_fname} > {minimap2_sam_annot_fname}"
    print(minimap2_cmd_annot)
    subprocess.run(minimap2_cmd_annot, shell=True, check=True)

hisat2 --score-min L,0,-2 --mp 2,2 -x /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Experiments/results/alignment/indices/hisat2/HXB2 -f -U /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Experiments/results/alignment/files/K03454/K03454.reads.fasta -S /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Experiments/results/alignment/files/K03454/K03454.hisat.woa.sam


126142 reads; of these:
  126142 (100.00%) were unpaired; of these:
    7555 (5.99%) aligned 0 times
    107999 (85.62%) aligned exactly 1 time
    10588 (8.39%) aligned >1 times
94.01% overall alignment rate


hisat2 --score-min L,0,-2 --mp 2,2 -x /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Experiments/results/alignment/indices/hisat2/HXB2_with_annotation -f -U /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Experiments/results/alignment/files/K03454/K03454.reads.fasta -S /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Experiments/results/alignment/files/K03454/K03454.hisat.wa.sam


126142 reads; of these:
  126142 (100.00%) were unpaired; of these:
    7556 (5.99%) aligned 0 times
    107843 (85.49%) aligned exactly 1 time
    10743 (8.52%) aligned >1 times
94.01% overall alignment rate
[M::mm_idx_gen::0.002*2.51] collected minimizers
[M::mm_idx_gen::0.004*2.62] sorted minimizers
[M::main::0.004*2.62] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.004*2.47] mid_occ = 10
[M::mm_idx_stat] kmer size: 15; skip: 5; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.004*2.43] distinct minimizers: 3114 (94.32% are singletons); average occurrences: 1.057; average spacing: 2.953; total length: 9719


minimap2 -a -x splice /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Creation/data/reference/K03455.1.fasta /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Experiments/results/alignment/files/K03454/K03454.reads.fasta > /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Experiments/results/alignment/files/K03454/K03454.minimap2.woa.sam


[M::worker_pipeline::1.235*2.77] mapped 126142 sequences
[M::main] Version: 2.28-r1209
[M::main] CMD: minimap2 -a -x splice /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Creation/data/reference/K03455.1.fasta /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Experiments/results/alignment/files/K03454/K03454.reads.fasta
[M::main] Real time: 1.237 sec; CPU: 3.424 sec; Peak RSS: 0.040 GB
[M::mm_idx_gen::0.001*3.86] collected minimizers
[M::mm_idx_gen::0.003*3.31] sorted minimizers
[M::main::0.003*3.30] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.003*3.06] mid_occ = 10
[M::mm_idx_stat] kmer size: 15; skip: 5; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.003*2.99] distinct minimizers: 3114 (94.32% are singletons); average occurrences: 1.057; average spacing: 2.953; total length: 9719


minimap2 --junc-bed /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Experiments/results/alignment/indices/minimap2/HXB2.junc.bed -a -x splice /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Creation/data/reference/K03455.1.fasta /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Experiments/results/alignment/files/K03454/K03454.reads.fasta > /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Experiments/results/alignment/files/K03454/K03454.minimap2.wa.sam


[M::worker_pipeline::1.243*2.77] mapped 126142 sequences
[M::main] Version: 2.28-r1209
[M::main] CMD: minimap2 --junc-bed /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Experiments/results/alignment/indices/minimap2/HXB2.junc.bed -a -x splice /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Creation/data/reference/K03455.1.fasta /ccb/salz8-3/avaraby1/HIV_Atlas/HIV_Atlas_Experiments/results/alignment/files/K03454/K03454.reads.fasta
[M::main] Real time: 1.245 sec; CPU: 3.445 sec; Peak RSS: 0.044 GB


In [12]:
# perhaps we should instead look for some real data and use that across all experiments instead
# including mapping rate
# and DGE/DTE/DTU